In [2]:
from sklearn.model_selection import KFold
from data import load_camera, load_cmfs, load_dataset_sfu,load_insitu,  msds_to_rgb, msds_to_xyz, load_dataset_csv
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer, RGBtoXYZNetwork
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import numpy as np
from colour import delta_E, XYZ_to_Lab

RANDOM_STATE = 42

SFU_FILE_PATH = 'data/reflect_db.reflect'  # SFU material reflectance database
INSITU_FILE_PATH = "data/insitu_dataset.csv"
CAVE_FOSTER_2002 = "data/cave_foster2002.csv"
CAMERA = 'sigma'


# Load the SFU dataset
sfu_dataset = load_dataset_sfu(SFU_FILE_PATH)
insitu_dataset = load_insitu(INSITU_FILE_PATH)
#cave_foster2002 = load_dataset_csv(CAVE_FOSTER_2002)
MSDS_TRAIN = load_camera(CAMERA)

cmfs, illuminant = load_cmfs()
y = msds_to_xyz(sfu_dataset, cmfs, illuminant)

X = msds_to_rgb(sfu_dataset,MSDS_TRAIN, illuminant)

In [3]:
# Initialize KFold with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Define models to evaluate
models = {
    "P-Spline-5": GAMOptimizer(lams=0.1, order=3, n_splines=5),
    "P-Spline-10": GAMOptimizer(lams=0.1, order=3, n_splines=10),
    "P-Spline-20": GAMOptimizer(lams=0.1, order=3, n_splines=20),

    # Define other models similarly...
}

# Dictionary to hold the results
would_clip = np.any(X * 2 > 1, axis=1)  # Change '> 1' if not using normalized RGB values
# Exclude these samples from both X and y
X = X[~would_clip]
y = y[~would_clip]
# Cross-validation loop
exposure_factors = [0.5, 1, 2]  # Simulate lower, same, and higher exposures
results = {model_name: {str(factor): [] for factor in exposure_factors} for model_name in models}

for train_index, test_index in kf.split(X):
    X_train, X_test_original = X[train_index], X[test_index]
    y_train, y_test_original = y[train_index], y[test_index]

    # Train and evaluate each model
    for name, model in models.items():

        model.fit(X_train, y_train)
        for factor in exposure_factors:
            # Simulate change in exposure by scaling X_test
            X_test = X_test_original * factor   

            # Clip the scaled RGB values to [1, 1, 1]
            X_test = np.clip(X_test, 0, 1)

            # Correspondingly scale y_test for LCC invariance test and clip to [0.9504, 1.0000, 1.0888]
            y_test = y_test_original
            y_test = np.clip(y_test, [0, 0, 0], [0.9504, 1.0000, 1.0888])

            # Make predictions
            xyz = model.predict(X_test) / factor
            # xyz = np.clip(xyz, [0, 0, 0], [0.9504, 1.0000, 1.0888])

            # Evaluate the model (using an appropriate metric, e.g., RMSE)
            lab_pred = XYZ_to_Lab(xyz)
            lab_tre = XYZ_to_Lab(y_test)
            score = np.mean(delta_E(lab_pred, lab_tre, method="CIE 1976"))

            #  Append the score
            results[name][str(factor)].append(score)    

average_scores = {model_name: {factor: np.mean(scores) for factor, scores in factors.items()} for model_name, factors in results.items()}

# Print or return the average scores
for model_name, scores_by_factor in average_scores.items():
    print(f"Model: {model_name}")
    for factor, score in scores_by_factor.items():
        print(f"  Exposure {factor}: Average Delta E = {score:.2f}")

0.347925616356
0.690299663476
1.34891732241
0.34727922648
0.697450666552
1.27180021236
0.346650130995
0.698541441532
1.67179743145
0.331713204896
0.659746153035
1.29855107436
0.331452602503
0.666703455015
1.23785264357
0.331005794311
0.668878311802
1.59675525531
0.35044840243
0.696806350125
1.36953086692
0.350090439768
0.70524002255
1.27197738478
0.349768749875
0.708238165113
1.50775768561
0.298887405355
0.597861512509
1.19971067218
0.298201491826
0.600016155999
1.22872055188
0.297864748
0.601953301281
1.54436160244
0.335552902099
0.667389783251
1.31384985027
0.33447679135
0.672791136093
1.24738568213
0.333395961941
0.678388237804
1.51708496751
Model: 
    "LCC": LinearRegression(fit_intercept=False),
    "RPCC2": Pipeline([
        ('transformer', PolynomialTransformer(degree=2, rp=True)),
        ('regressor', LinearRegression(fit_intercept=False))
    ]),
    "RPCC3": Pipeline([
        ('transformer', PolynomialTransformer(degree=3, rp=True)),
        ('regressor', LinearRegression

In [4]:
%history

from sklearn.model_selection import KFold
from data import load_camera, load_cmfs, load_dataset_sfu,load_insitu,  msds_to_rgb, msds_to_xyz, load_dataset_csv
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer, RGBtoXYZNetwork
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import numpy as np
from colour import delta_E, XYZ_to_Lab

RANDOM_STATE = 42

SFU_FILE_PATH = 'data/reflect_db.reflect'  # SFU material reflectance database
INSITU_FILE_PATH = "data/insitu_dataset.csv"
CAVE_FOSTER_2002 = "data/cave_foster2002.csv"
CAMERA = 'sigma'


# Load the SFU dataset
sfu_dataset = load_dataset_sfu(SFU_FILE_PATH)
insitu_dataset = load_insitu(INSITU_FILE_PATH)
cave_foster2002 = load_dataset_csv(CAVE_FOSTER_2002)
MSDS_TRAIN = load_camera(CAMERA)

cmfs, illuminant = load_cmfs()
y = msds_to_xyz(cave_foster2002, cmfs, illuminant)

X = msds_to_rgb(cave_foster2002,MSDS_TRAIN, illuminant)
from sklearn.model_selection import KFold
from data impor